In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

In [3]:
# 주중-월ver 파일
now = datetime.now()
f_date = now.strftime('%Y%m%d')
df = pd.read_csv(f'data/{f_date}_SUBWAY_MONTH_주중_월ver.csv', encoding='utf-8', sep=',')

``` 1. 호선별 파일 분리 -> 2. 월별 파일 분리```

In [25]:
# 반복문 사용 테스트 코드 --> 코드 파일에 맞게 변형해서 적용하기
df1 = df[df['호선명']=='1호선'].copy()
df2 = df[df['호선명']=='경부선'].copy()
df3 = df[df['호선명']=='경원선'].copy()
df_list = { '1호선': df1, '경부선': df2, '경원선': df3}
for line, frame in df_list.items():
    frame = df[df['호선명']==line].copy()
    frame = frame[['사용월','호선명','지하철역', 
                '06시-07시 승차인원','07시-08시 승차인원','08시-09시 승차인원', 
                '17시-18시 승차인원','18시-19시 승차인원','19시-20시 승차인원', 
                '06시-07시 하차인원','07시-08시 하차인원','08시-09시 하차인원', 
                '17시-18시 하차인원','18시-19시 하차인원','19시-20시 하차인원']].copy()
    frame['출근시간 승차인원'] = frame.iloc[:,3:6].sum(axis=1)
    frame['퇴근시간 승차인원'] = frame.iloc[:,6:9].sum(axis=1)
    frame['출근시간 하차인원'] = frame.iloc[:,9:12].sum(axis=1)
    frame['퇴근시간 하차인원'] = frame.iloc[:,12:15].sum(axis=1)
    frame = frame[['사용월', '호선명', '지하철역', '출근시간 승차인원', '출근시간 하차인원', '퇴근시간 승차인원', '퇴근시간 하차인원']]
    frame.to_csv(f'data/{line}.csv',index=False,encoding='utf-8')

,사용월,호선명,지하철역,승차총승객수,하차총승객수,승하차총승객수,주중주말
0,202101,일산선,백석,2724,2825,5549,주중
1,202101,일산선,마두,1638,1652,3290,주중
2,202101,3호선,을지로3가,1024,1009,2033,주중
3,202101,3호선,종로3가,1255,1632,2887,주중
4,202101,3호선,안국,2934,3029,5963,주중
...,...,...,...,...,...,...,...
219274,202112,6호선,월곡,10214,9924,20138,주중
219275,202112,6호선,상월곡,5619,4842,10461,주중
219276,202112,6호선,돌곶이,8265,7811,16076,주중
219277,202112,6호선,석계,11189,11335,22524,주중


In [ ]:
################ 호선 병합 - 파일 단위일 때 merge하기 #############
path = 'data/main/temp_files/'
line_info = [
    ([f'{path}1호선.csv', f'{path}경부선.csv', f'{path}경원선.csv', f'{path}경인선.csv', f'{path}장항선.csv'], '1호선'),
    ([f'{path}2호선.csv'], '2호선'),
    ([f'{path}3호선.csv', f'{path}일산선.csv'], '3호선'),
    ([f'{path}4호선.csv', f'{path}과천선.csv', f'{path}안산선.csv'], '4호선'),
    ([f'{path}5호선.csv'], '5호선'),
    ([f'{path}6호선.csv'], '6호선'),
    ([f'{path}7호선.csv'], '7호선'),
    ([f'{path}8호선.csv'], '8호선'),
    ([f'{path}9호선.csv', f'{path}9호선2단계.csv', f'{path}9호선2~3단계.csv'], '9호선'),
    ([f'{path}수인선.csv', f'{path}분당선.csv'], '수인분당선'),
    ([f'{path}경의선.csv', f'{path}중앙선.csv'], '경의중앙선')

]
line_list = []
for df_list, line_name in line_info:
    df_copies = []
    for file in df_list:
        df = pd.read_csv(file)
        df_copies.append(df.copy())
    result = pd.concat(df_copies, axis=0)
    result = result.reset_index(drop=True)
    result.호선명 = line_name
    cols = list(result.columns)[:3]
    target = list(result.columns)[3:]
    res = result.groupby(cols)[target].agg('sum').reset_index()
    line_list.append(res)
    # res.to_csv(f'data/main//lines/{line_name}.csv', index=False)
final = pd.concat(line_list,axis=0)


In [ ]:
################ 호선 병합 - 컬럼 내 데이터 단위 합치기ver #############
line_info = [
    df.replace({'호선명': ['1호선','경부선','경원선','경인선','장항선']}, '1호선', inplace=True),
    df.replace({'호선명':'2호선'},'2호선' ,inplace=True),
    df.replace({'호선명':'5호선'},'5호선' ,inplace=True),
    df.replace({'호선명':'7호선'},'7호선' ,inplace=True),
    df.replace({'호선명':'8호선'},'8호선' ,inplace=True),
    df.replace({'호선명': ['3호선','일산선']}, '3호선' ,inplace=True),
    df.replace({'호선명': ['4호선','과천선','안산선']}, '4호선' ,inplace=True),
    df.replace({'호선명': ['9호선','9호선2~3단계']}, '9호선' ,inplace=True),
    df.replace({'호선명': ['수인선','분당선']}, '수인분당선' ,inplace=True),
    df.replace({'호선명': ['경의선','용산선','중앙선']}, '경의중앙선' ,inplace=True)
]

for df_list, line_name in line_info:
    df_copies = []
    for file in df_list:
        df = pd.read_csv(file)
        df_copies.append(df.copy())
    result = pd.concat(df_copies, axis=0)
    result = result.reset_index(drop=True)
    result.호선명 = line_name
    cols = list(result.columns)[:3]
    target = list(result.columns)[3:]
    res = result.groupby(cols)[target].agg('sum').reset_index()
    res.to_csv(f'./lines/{line_name}.csv', index=False)


In [ ]:
######################## 호선 분리 1 ##########################

# # 서울지역 1호선, 경부선, 경원선, 구로에서 인천방향은 제외함

# 1호선:서울지하철1호선(청량리~서울역), 경원선(소요산~청량리), 경부선(서울~천안/서동탄/광명), 장항선(천안~신창), 경인선(구로~인천)
df_list = ['./1호선.csv', './경부선.csv', './경원선.csv', './경인선.csv', './장항선.csv']
df_copies = []

for file in df_list:
    df = pd.read_csv(file)
    df_copies.append(df.copy())

result = pd.concat(df_copies, axis=0)
result = result.reset_index(drop=True)
result.호선명 = '1호선'

cols = list(result.columns)[:3]
target = list(result.columns)[3:]
res = result.groupby(cols)[target].agg('sum').reset_index()
res.to_csv('./lines/1호선.cvs',index=False)


In [ ]:
# 1호선:서울지하철1호선(청량리~서울역), 경원선(소요산~청량리), 경부선(서울~천안/서동탄/광명), 장항선(천안~신창), 경인선(구로~인천)
# 2호선, 5호선, 6호선, 7호선, 8호선
# 3호선: 서울지하철3호선(지축~오금), 일산선(대화~지축)
# 4호선: 서울지하철4호선(당고개~남태령), 과천선(남태령~금정), 안산선(금정~오이도)
# 9호선: 9호선, 9호선(2~3)단계
# 수인분당선: 수인선(인천~수원), 분당선()
# 경의중앙선: 경의선, 중앙선
# 우이신설선
# ...그 외 호선 파일들 추가할 것

In [5]:
# 한번에 실행하는 코드
# 서울지역 1호선, 경부선, 경원선, 구로에서 인천방향은 제외함
# 시간 나누는 기준 정해야함
# 02시 이후로는 이용객이 거의 없어서 총합에서 제외함
# 반복문 사용하기 위해 dataframe 선언
df1 = df[df['호선명']=='1호선'].copy()
df2 = df[df['호선명']=='경부선'].copy()
df3 = df[df['호선명']=='경원선'].copy()
df4 = df[df['호선명']=='경인선'].copy()
df5 = df[df['호선명']=='장항선'].copy()
# 사용할 dictionary 만듬
df_dict = { '1호선': df1, '경부선': df2, '경원선': df3, '경인선': df4, '장항선': df5}
# 반복문 사용해서 데이터 가공
# 저장될 csv 파일 이름은 호선명, 인코딩 방식은 utf-8
for line, frame in df_dict.items():
    # frame = df[df['호선명']==line].copy()
    frame['새벽 승차인원'] = frame.loc[:,['04시-05시 승차인원','05시-06시 승차인원']].sum(axis=1)
    frame['새벽 하차인원'] = frame.loc[:,['04시-05시 하차인원','05시-06시 하차인원']].sum(axis=1)

    frame['출근시간 승차인원'] = frame.loc[:,['06시-07시 승차인원','07시-08시 승차인원','08시-09시 승차인원']].sum(axis=1)
    frame['09-16시 승차인원'] = frame.loc[:,['09시-10시 승차인원','10시-11시 승차인원','11시-12시 승차인원',
                                            '12시-13시 승차인원','13시-14시 승차인원','14시-15시 승차인원','16시-17시 승차인원']].sum(axis=1)
    frame['퇴근시간 승차인원'] = frame.loc[:,['17시-18시 승차인원','18시-19시 승차인원','19시-20시 승차인원']].sum(axis=1)
    frame['야간 승차인원'] = frame.loc[:,['20시-21시 승차인원','21시-22시 승차인원','22시-23시 승차인원',
                                        '23시-24시 승차인원','00시-01시 승차인원','01시-02시 승차인원']].sum(axis=1)
    frame['출근시간 하차인원'] = frame.loc[:,['06시-07시 하차인원','07시-08시 하차인원','08시-09시 하차인원']].sum(axis=1)
    frame['퇴근시간 하차인원'] = frame.loc[:,['17시-18시 하차인원','18시-19시 하차인원','19시-20시 하차인원']].sum(axis=1)
    frame['09-16시 하차인원'] = frame.loc[:,['09시-10시 하차인원','10시-11시 하차인원','11시-12시 하차인원',
                                            '12시-13시 하차인원','13시-14시 하차인원','14시-15시 하차인원','16시-17시 하차인원']].sum(axis=1)
    frame['야간 하차인원'] = frame.loc[:,['20시-21시 하차인원','21시-22시 하차인원','22시-23시 하차인원',
                                        '23시-24시 하차인원','00시-01시 하차인원','01시-02시 하차인원']].sum(axis=1)
    frame['총 승차인원'] = frame.loc[:,['새벽 승차인원','출근시간 승차인원','09-16시 승차인원','야간 승차인원']].sum(axis=1)
    frame['총 하차인원'] = frame.loc[:,['새벽 하차인원','출근시간 하차인원','09-16시 하차인원','야간 하차인원']].sum(axis=1)
    frame = frame[['사용월', '호선명', '지하철역', '출근시간 승차인원', '출근시간 하차인원', 
                    '09-16시 승차인원', '09-16시 하차인원', '퇴근시간 승차인원', '퇴근시간 하차인원',
                    '새벽 승차인원','새벽 하차인원','야간 승차인원', '야간 하차인원','총 승차인원','총 하차인원']]
    frame.to_csv(f'data/main/{line}.csv',index=False,encoding='utf-8')

In [ ]:
# 사용할 노선과 합쳐야 하는 노선 목록(중복역은 합계 후 중복 삭제예정)
# 1호선:서울지하철1호선(청량리~서울역), 경원선(소요산~청량리), 경부선(서울~천안/서동탄/광명), 장항선(천안~신창), 경인선(구로~인천)
# 2호선, 5호선, 6호선, 7호선, 8호선
# 3호선: 서울지하철3호선(지축~오금), 일산선(대화~지축)
# 4호선: 서울지하철4호선(당고개~남태령), 과천선(남태령~금정), 안산선(금정~오이도)
# 9호선: 9호선, 9호선(2~3)단계
# 수인분당선: 수인선(인천~수원), 분당선()
# 공항철도
# 신림선, 우이신설선
# 신분당선 데이터 없음, 김포 골드라인 없음, 용인 에버 없음

# 호선 분리하기 ex) 1호선, 경원선, 수인선 ......

# 월별 평일 승하차 각각 합계, 월별 주말 승하차 각각 합계

# 월별로 처리한 데이터 csv로 각 년도 폴더에 저장



In [11]:
# 호선명 리스트로 추출해서 사용 
lines = df.호선명.unique().tolist()
df_dict = {line: df[df['호선명'] == line].copy() for line in lines}
for line, frame in df_dict.items():
    # frame = df[df['호선명']==line].copy()
    frame['새벽 승차인원'] = frame.loc[:,['04시-05시 승차인원','05시-06시 승차인원']].sum(axis=1)
    frame['새벽 하차인원'] = frame.loc[:,['04시-05시 하차인원','05시-06시 하차인원']].sum(axis=1)

    frame['출근시간 승차인원'] = frame.loc[:,['06시-07시 승차인원','07시-08시 승차인원','08시-09시 승차인원']].sum(axis=1)
    frame['09-16시 승차인원'] = frame.loc[:,['09시-10시 승차인원','10시-11시 승차인원','11시-12시 승차인원',
                                            '12시-13시 승차인원','13시-14시 승차인원','14시-15시 승차인원','16시-17시 승차인원']].sum(axis=1)
    frame['퇴근시간 승차인원'] = frame.loc[:,['17시-18시 승차인원','18시-19시 승차인원','19시-20시 승차인원']].sum(axis=1)
    frame['야간 승차인원'] = frame.loc[:,['20시-21시 승차인원','21시-22시 승차인원','22시-23시 승차인원',
                                        '23시-24시 승차인원','00시-01시 승차인원','01시-02시 승차인원']].sum(axis=1)
    frame['출근시간 하차인원'] = frame.loc[:,['06시-07시 하차인원','07시-08시 하차인원','08시-09시 하차인원']].sum(axis=1)
    frame['퇴근시간 하차인원'] = frame.loc[:,['17시-18시 하차인원','18시-19시 하차인원','19시-20시 하차인원']].sum(axis=1)
    frame['09-16시 하차인원'] = frame.loc[:,['09시-10시 하차인원','10시-11시 하차인원','11시-12시 하차인원',
                                            '12시-13시 하차인원','13시-14시 하차인원','14시-15시 하차인원','16시-17시 하차인원']].sum(axis=1)
    frame['야간 하차인원'] = frame.loc[:,['20시-21시 하차인원','21시-22시 하차인원','22시-23시 하차인원',
                                        '23시-24시 하차인원','00시-01시 하차인원','01시-02시 하차인원']].sum(axis=1)
    frame['총 승차인원'] = frame.loc[:,['새벽 승차인원','출근시간 승차인원','09-16시 승차인원','야간 승차인원']].sum(axis=1)
    frame['총 하차인원'] = frame.loc[:,['새벽 하차인원','출근시간 하차인원','09-16시 하차인원','야간 하차인원']].sum(axis=1)
    frame = frame[['사용월', '호선명', '지하철역', '출근시간 승차인원', '출근시간 하차인원', 
                    '09-16시 승차인원', '09-16시 하차인원', '퇴근시간 승차인원', '퇴근시간 하차인원',
                    '새벽 승차인원','새벽 하차인원','야간 승차인원', '야간 하차인원','총 승차인원','총 하차인원']]
    
    frame.loc[(frame['호선명'] == '2호선') & (frame['지하철역'] == '신천'), '지하철역'] = '잠실새내'

    frame.to_csv(f'../data/main/temp_files/{line}.csv',index=False,encoding='utf-8')